In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
!!pip install spacy-transformers

['Collecting spacy-transformers',
 '  Downloading spacy_transformers-1.3.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)',
 'Requirement already satisfied: spacy<4.1.0,>=3.5.0 in /usr/local/lib/python3.10/dist-packages (from spacy-transformers) (3.7.6)',
 'Collecting transformers<4.37.0,>=3.4.0 (from spacy-transformers)',
 '  Downloading transformers-4.36.2-py3-none-any.whl.metadata (126 kB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.6 MB/s eta 0:00:00',
 'Requirement already satisfied: torch>=1.8.0 in /usr/local/lib/python3.10/dist-packages (from spacy-transformers) (2.4.1+cu121)',
 'Requirement already satisfied: srsly<3.0.0,>=2.4.0 in /usr/local/lib/python3.10/dist-packages (from spacy-transformers) (2.4.8)',
 'Collecting spacy-alignments<1.0.0,>=0.7.2 (from spacy-transformers)',
 '  Downloading spacy_alignments-0.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.7 kB)',
 'Requirement already satisfied: 

In [6]:
import spacy_transformers

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [9]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



In [13]:
# !python3 -m spacy download en_core_web_trf
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:000:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
# Load spaCy's English model
nlp = spacy.load("en_core_web_lg")

In [31]:


def preprocess_text(text):
    """Normalize and clean text data."""
    return text.strip().lower()

def extract_features(text):
    """Extract syntactic and contextual features using spaCy."""
    doc = nlp(text)
    features = {
        "root_verb": None,
        "object_entity": None,
        "dependency_relations": [],
        "modifiers": []
    }

    for token in doc:
        # Identify the root verb (e.g., "disconnect")
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            features["root_verb"] = token.lemma_
            
            # Find associated entities (objects or subjects)
            for child in token.children:
                if child.dep_ in {"dobj", "nsubj", "prep"}:
                    features["object_entity"] = child.text
                
                # Collect dependency relations and modifiers
                features["dependency_relations"].append((child.dep_, child.text))
                if child.dep_ == "advmod":
                    features["modifiers"].append(child.text)
    
    return features

# Example dataset (replace this with actual labeled data)
data = pd.DataFrame({
    "utterance": [
        "I want to disconnect my service.",
        "I'm concerned this call might get disconnected.",
        "My service was disconnected because I didn't pay the bill."
    ],
    "label": [1, 2, 3]  # 1: Cancel service, 2: Call might disconnect, 3: Service disconnected due to non-payment
})

# Preprocess the data
data["utterance"] = data["utterance"].apply(preprocess_text)

# Extract features
data["features"] = data["utterance"].apply(extract_features)

# Prepare the dataset for training
vectorizer = DictVectorizer(sparse=False)

new_features = []
for x in data["features"]:
    new_features.append(convert_to_string_dict(x))
    
X = vectorizer.fit_transform(new_features)
y = data["label"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Example usage
def classify_utterance(utterance):
    """Classify a new utterance."""
    preprocessed = preprocess_text(utterance)
    features = extract_features(preprocessed)
    print(features)
    features_vectorized = vectorizer.transform([convert_to_string_dict(features)])
    prediction = clf.predict(features_vectorized)
    return prediction[0]

# Test with a new utterance
new_utterance = "This call might get disconnected accidentally."
print("Predicted Class:", classify_utterance(new_utterance))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

{'root_verb': 'disconnect', 'object_entity': None, 'dependency_relations': [('nsubjpass', 'call'), ('aux', 'might'), ('auxpass', 'get'), ('advmod', 'accidentally'), ('punct', '.')], 'modifiers': ['accidentally']}
Predicted Class: 2


In [33]:
import ast

def convert_to_string_dict(input_dict):
    
    output_dict = {}
    
    for key, value in input_dict.items():
        if isinstance(value, list):  # Handle lists
            if len(value) > 0 and isinstance(value[0], tuple):  # If list contains tuples
                for tuple_item in value:
                    tuple_key, tuple_value = tuple_item
                    # If the key already exists, append the value; otherwise, set it
                    if tuple_key in output_dict:
                        output_dict[tuple_key] += f", {tuple_value}"
                    else:
                        output_dict[tuple_key] = tuple_value
            else:
                # If list is empty or doesn't contain tuples, join as string
                output_dict[key] = ", ".join(map(str, value)) if value else ""
        else:
            output_dict[key] = str(value)  # Convert non-list values to string
    
    return output_dict

# Example input with multiple `nsubj` and `xcomp`
input_string = {'root_verb': 'want', 'object_entity': 'i', 'dependency_relations': [('nsubj', 'i'), ('xcomp', 'disconnect'), ('nsubj', 'they'), ('punct', '.'), ('xcomp', 'terminate')], 'modifiers': []}

# Convert and print the result
result = convert_to_string_dict(input_string)
print(result)


{'root_verb': 'want', 'object_entity': 'i', 'nsubj': 'i, they', 'xcomp': 'disconnect, terminate', 'punct': '.', 'modifiers': ''}
